In [7]:
import threading
import socket
import pickle

In [8]:
import time
import hashlib
import json
import uuid

NANOSECONDS = 1
MICROSECONDS = 1000 * NANOSECONDS
MILLISECONDS = 1000 * MICROSECONDS
SECONDS = 1000 * MILLISECONDS

MINE_RATE = 4 * SECONDS

STARTING_BALANCE = 1000

MINING_REWARD = 50
MINING_REWARD_INPUT = { 'address': '*--official-mining-reward--*' }

In [9]:
'''TRANSACTION POOL'''
class TransactionPool:
    def __init__(self):
        self.transaction_map = {}

    def set_transaction(self, transaction):
        """
        Set a transaction in the transaction pool.
        """
        self.transaction_map[transaction.id] = transaction


    def existing_transaction(self, address):
        """
        Find a transaction generated by the address in the transaction pool
        """
        for transaction in self.transaction_map.values():
            if transaction.input['address'] == address:
                return transaction
    
    def transaction_data(self):
        """
        Return the transactions of thje transaction pool represented in their
        json serialized form.
        """
        return list(map(
            lambda transaction: transaction.to_json(),
            self.transaction_map.values()
        ))

    def clear_blockchain_transactions(self, blockchain):
        """
        Delete blockchain recorded transactions from the transaction pool.
        """
        for block in blockchain.chain:
            for transaction in block.data:
                try:
                    del self.transaction_map[transaction['id']]
                except KeyError:
                    pass

In [10]:
class ManagerPeer:
    def __init__(self, ip):
        self.peer_set = set()
        self.MANAGER_IP = ip
        self.MANAGER_PORT = 8817
        self.server_port = 5517
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_thread = threading.Thread(target=self.start_server)
        self.server_thread.daemon = True
        self.server_thread.start()
        self.stop_server = False
        self.transaction_pool = TransactionPool()
        
    def start_server(self):
        self.server_socket.bind((self.MANAGER_IP, self.MANAGER_PORT))
        self.server_socket.listen(100)
        while True:
            if self.stop_server:
                break

            client_peer, address = self.server_socket.accept()
            print(f"incoming connection from {address}")
            recieved_request = client_peer.recv(4096).decode('utf-8')            
            ip = recieved_request.split()[1]

            if recieved_request.startswith("add"):
                self.inform_peers(recieved_request)
                client_peer.send(pickle.dumps(self.peer_set))
                client_peer.send(pickle.dumps(self.transaction_pool))
                self.peer_set.add(ip)
                print(f"added {ip}")
                
            elif recieved_request.startswith('remove'):
                self.peer_set.remove(ip)
                self.inform_peers(recieved_request)
                print(f"removed {ip}")
            
            elif recieved_request.startswith('update-transactionpool'):
                new_transactionpool = client_peer.recv(4096)
                self.transaction_pool = pickle.loads(new_transactionpool)
            
            client_peer.close()
        self.server_socket.close()
    
    def _stop_server(self):
        self.stop_server = True
    
    def inform_peers(self, message):
        operation, new_peer = message.split()
        for peer in self.peer_set:
            client_socket = socket.socket()
            client_socket.settimeout(30)
            try:
                client_socket.connect((peer, self.server_port))
                client_socket.send(f"{operation}-peer {new_peer}".encode('utf-8'))
                print(f"Sent to {peer}")
                
            except Exception as e:
                print(f"{operation}-peer {peer}")
                print(e)
            finally:
                client_socket.close()
    def __repr__(self):
        return f'{self.MANAGER_IP} {self.MANAGER_PORT}\nPeers: '+' '.join(self.peer_set)
    
    

In [113]:
manager_peer = ManagerPeer('192.168.1.109')

incoming connection from ('192.168.1.110', 51304)
added 192.168.1.110
incoming connection from ('192.168.1.106', 60836)
Sent to 192.168.1.110
added 192.168.1.106
incoming connection from ('192.168.1.111', 51248)
Sent to 192.168.1.106
Sent to 192.168.1.110
added 192.168.1.111


In [109]:
print(manager_peer.peer_set) 

set()


In [87]:
manager_peer._stop_server()

In [94]:
manager_peer

192.168.1.109 8811
Peers: 192.168.1.106 192.168.1.111 192.168.1.110

In [57]:
s = set(['a','b'])

In [59]:
'hey'+' '.join(s)

'heyb a'